## Constant-depth circuits for simulating the XY model using Qsearch

In this notebook, we demonstrate how to generate the constant-depth circuits for dynamic simulation of the XY model using the circuit synthesis software QSearch, which is part of the **[BQSKit](https://bqskit.lbl.gov/)** tookit.  More details on our constant-depth circuits can be found in our **[paper](https://arxiv.org/abs/2103.07429)**.  We also prepare analgous circuits using IBM's general purpose compiler and show how our circuits perform better on currently available quantum computers.

### Import necessary libraries
First we download the necessary libraries.  See instructions for downloading **[numpy](https://numpy.org/install/)**, **[matplotlib](https://matplotlib.org/stable/users/installing.html)**, **[QSearch](https://github.com/BQSKit/qsearch)**, or **[Qiskit](https://qiskit.org/documentation/install.html)** if you don't already have them installed on your compuer.

In [ ]:
#import necessary libraries
#libraries for notebook
import numpy as np
import matplotlib.pyplot as plt
#QSearch import
import qsearch
from qsearch.gates import *
from qsearch.assemblers import *
from qsearch import multistart_solvers, utils, options, leap_compiler, post_processing, assemblers
from qsearch.defaults import standard_defaults, standard_smart_defaults
#Qiskit import
import qiskit as qk
from qiskit.tools.monitor import job_monitor
from qiskit import Aer, IBMQ, execute

### System parameters
Next, we define the parameters of the system and the simulation we wish to run. Here, we will be performing a quantum quench of the XY model.  Set the number of desired qubits, the coupling constants Jx and Jy of the XY model, the time-step size, the number of time-steps, and the number of shots for performing experiments on the quantum simulator or the real quantum hardware.

In [ ]:
#Define simulation parameters
N = 3 #number of spins/qubits
#XY model Simulation parameters
Jx = -1.0
Jy = -1.0
delta_t = 0.025 #time-step size
num_steps = 40 #number of time-steps, we start with a small number for speed of demonstration
shots = 8192 #number of shots for circuit execution

### Generating the high-level circuits
Next, we generate the high-level circuits for simulating the dynamics of the XY model.  These circuits will be used to create our constant-depth circuits, as well as used with the IBM general-purpose circuit compiler for comparison.

In [ ]:
#create the circuit to execute the time-evolution operator for a given time-step
def evolution_circuit_XY(num_time_steps, Jx, Jy, delta_t, N):
    hbar = 0.658212    # eV*fs
    time_evol_circuit = qk.QuantumCircuit(N)
    #define rotation angles for gates in circuit
    psiX = -2.0*Jx*delta_t/hbar
    psiY = -2.0*Jy*delta_t/hbar

    for step in range(num_time_steps):
        #implement XX operator
        for q in range(0,N-1):
            time_evol_circuit.h(q)
            time_evol_circuit.h(q+1)
            time_evol_circuit.cx(q,q+1)
            time_evol_circuit.rz(psiX,q+1)
            time_evol_circuit.cx(q,q+1)
            time_evol_circuit.h(q)
            time_evol_circuit.h(q+1)
        #implement YY operator
        for q in range(0,N-1):
            time_evol_circuit.rx(-np.pi/2, q)
            time_evol_circuit.rx(-np.pi/2,q+1)
            time_evol_circuit.cx(q,q+1)
            time_evol_circuit.rz(psiY,q+1)
            time_evol_circuit.cx(q,q+1)
            time_evol_circuit.rx(np.pi/2, q)
            time_evol_circuit.rx(np.pi/2,q+1)
    return time_evol_circuit

#draw circuit for time-step 1
#circ = evolution_circuit_XY(1,Jx,Jy,N)
#circ.draw()

In [ ]:
#Create set of final circuits for quantum quench simulations
circuits = []
for i in range(0, num_steps+1):
    #XY model
    circuits.append(evolution_circuit_XY(i,Jx,Jy,delta_t,N))

### Define functions for generating the constant-depth circuit structure

In [ ]:
def make_matchgate():
    rz = ZGate()
    rx = XGate()
    cnot = CNOTGate()
    I = IdentityGate()

    RZ_layer = KroneckerGate(rz,rz)
    RX_layer = KroneckerGate(rx,rx)
    RXRZ_layer = KroneckerGate(rx, rz)
    matchgate = ProductGate(RZ_layer, RX_layer, cnot, RXRZ_layer, cnot, RX_layer, RZ_layer)
    return matchgate
    
def make_layertype1(N):
    I = IdentityGate()
    for i in range(int(N/2)):
        if (i==0):
            layer = make_matchgate()
        else:
            layer = KroneckerGate(layer,make_matchgate())
    if (N%2 != 0): 
        layer = KroneckerGate(layer, I)
    return layer
        
def make_layertype2(N):
    I = IdentityGate()
    layer = I
    #N even
    if (N%2==0):
        for _ in range(int(N/2)-1):
            layer = KroneckerGate(layer,make_matchgate())
        layer = KroneckerGate(layer,I)
    #N odd
    else:
        for _ in range(int(N/2)):
            layer = KroneckerGate(layer,make_matchgate())
    return layer

def make_MGC(N):
    """
    Make matchgate circuit for N qubits.
    Args:
        N (int): The number of spins.
    Returns
        circuit (ProductGate): Circuit of QSearch gates.
    """
    for l in range(N):
        #add layer_type1
        if (l%2 == 0):
            if(l==0):
                circuit = make_layertype2(N)
            else:
                circuit = ProductGate(circuit, make_layertype2(N))
        #add layer_type2
        else:
            circuit = ProductGate(circuit, make_layertype1(N))
    return circuit

### Generate the constant-depth circuits
The constant-depth circuits are generating by computing the target unitary matrix from the high-level circuits and the constant-depth circuit structure.  Given the target unitary and the constant-depth circuit structure, QSearch can optimize the parameters in the constant-depth circuit, which are written to a file for future use.

In [ ]:
#make constant-depth circuits
unitary_sim = Aer.get_backend('unitary_simulator')
for t in range(num_steps+1):
    #get target unitary for the given timestep
    job = execute(circuits[t], unitary_sim)
    result = job.result()
    target_unitary = result.get_unitary(circuits[t], decimals=12)
    #get constant-depth circuit structure for N qubits
    circ_struct = make_MGC(N)
    #optimize parameters of circuit
    # use the multistart solver, may want to increase the number of starts for more qubits, but that will also be slower
    solv = multistart_solvers.MultiStart_Solver(24)
    # set up some options
    opts = qsearch.Options()
    opts.target = target_unitary
    #opts.gateset = gateset
    opts.set_defaults(**standard_defaults)
    opts.set_smart_defaults(**standard_smart_defaults)
    # optimize the circuit structure (circ_struct) for target U
    # returns the calculated matrix and the vector of parameters
    dist = 1
    # run a few times to make sure we find the correct solution
    for _ in range(5):
        mat, vec = solv.solve_for_unitary(circ_struct, opts)
        dist_new = utils.matrix_distance_squared(mat, target_unitary)
        print(dist_new)
        if dist_new < dist:
            dist = dist_new
        if dist < 1e-10:
            break

    print(f'For timestep {t} got distance {dist}')
    #get final circuit

    result_dict = {}
    result_dict["structure"] = circ_struct
    result_dict["parameters"] = vec
    
    opts.assemblydict=assemblydict_ibmopenqasm
    out = opts.assembler.assemble(result_dict, opts)
    #write results to file
    with open(f'{N}q_XY_timestep{t}.qasm', "w") as wfile:
        wfile.write(out)


In [ ]:
#prepare constant depth circuits to run on quantum backends
q_regs = qk.QuantumRegister(N, 'q')
c_regs = qk.ClassicalRegister(N, 'c')
cd_circuits = []
for t in range(num_steps+1):
    total_circ = qk.QuantumCircuit(q_regs, c_regs)
    #initialize to the ground state of ZZ, i.e., the Neel state
    for q in range(N):
        if(q%2 != 0):
            total_circ.x(q_regs[q])
    total_circ.barrier()
    circ = qk.QuantumCircuit.from_qasm_file(f'{N}q_XY_timestep{t}.qasm')
    total_circ.compose(circ, inplace=True)
    total_circ.measure(q_regs, c_regs)
    cd_circuits.append(total_circ)

### Generate the IBM-compiled circuits
For comparison, we generate circuits using IBM's general-purpose quantum circuit compiler.  These circuits will not be constant in depth with increasing numbers of time-steps.

In [ ]:
#For comparison, create circuits using IBM compiler
q_regs = qk.QuantumRegister(N, 'q')
c_regs = qk.ClassicalRegister(N, 'c')
ibm_circuits = []
for t in range(num_steps+1):
    total_circ = qk.QuantumCircuit(q_regs, c_regs)
    #initialize to the ground state of ZZ, i.e., the Neel state
    for q in range(N):
        if(q%2 != 0):
            total_circ.x(q_regs[q])
    #add evolution circuit
    total_circ.compose(circuits[t], inplace=True)
    total_circ.measure(q_regs, c_regs)
    ibm_circuits.append(total_circ)

### Compare sizes of the consant-depth and IBM-compiled circuits
Chose any time-step t to see how the volume of our constant-depth circuits compare to those generated by IBM's general-purpose compiler.

In [ ]:
#compare lengths of IBM vs constant-depth circuits for time-step t
t = 9
print(len(circuits[t]))
print(len(cd_circuits[t]))

### Connect to IBM quantum backends over the cloud

In [ ]:
#make connection to IBM to run constant-depth circuits on quantum backend
#!! If this is your first time running this notebook get you IBM accounts's API key and save your account!!
#qk.IBMQ.save_account('your_API_key_here')
qk.IBMQ.load_account()
simulator = Aer.get_backend('qasm_simulator')

#check for available quantum processor backend
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

In [ ]:
#select one of the available quantum backend
backend = provider.get_backend('ibmq_athens')

### Transpile all circuits for the target quantum backend

In [ ]:
#compile constant-depth circuits for the simulator
cd_circs_sim = qk.transpile(cd_circuits, backend=simulator, optimization_level=3)

In [ ]:
#compile constant-depth circuits for real quantum backend
cd_circs_qp = qk.transpile(cd_circuits, backend=backend, optimization_level=3)

In [ ]:
#compile ibm circuits for the simulator
ibm_circs_sim = qk.transpile(ibm_circuits, backend=simulator, optimization_level=3)

In [ ]:
#compile ibm circuits for real quantum backend
ibm_circs_qp = qk.transpile(ibm_circuits, backend=backend, optimization_level=3)

### Execute all circuits on desired qunatum backend

In [ ]:
#Run constant-depth circuits on simulator
cd_sim_results = execute(cd_circs_sim, simulator, shots=shots).result()

In [ ]:
#Run constant-depth circuits on quantum processor
cd_job = qk.execute(cd_circs_qp, backend=backend, shots=shots)
job_monitor(cd_job)

In [ ]:
#For comparison run IBM circuits on simulator
ibm_sim_results = execute(ibm_circs_sim, simulator, shots=shots).result()

In [ ]:
#For comparison run IBM circuits on real quantum processor
ibm_job = qk.execute(ibm_circs_qp, backend=backend, shots=shots)
job_monitor(ibm_job)

### Post-processing
To post-process the results from the quantum backends, we define a function to compute the staggered magnetization for each time-step from the results of the quantum backend. 

In [ ]:
#Define post-processing function
def staggered_magnetization(result: dict, shots: int):
    """Compute staggered magnetization from results of qk.execution.
    Args:
    - result (dict): a dictionary with the counts for each qubit, see qk.result.result module
    - shots (int): number of trials
    Return:
    - average_sm (float)
    """
    sm_val = 0
    for spin_str, count in result.items():
        #reverse big-endian and little endian due to IBM convention
        spin_str_rev = spin_str[::-1]
        spin_int = [1 - 2 * float(s) for s in spin_str_rev]
        for i in range(len(spin_int)):
            spin_int[i] = spin_int[i]*(-1)**i
        sm_val += (sum(spin_int) / len(spin_int)) * count
    average_sm = sm_val/shots
    return average_sm

In [ ]:
#Simulator post-processing for constant-depth circuits
avg_sm_cd_sim = []
for c in cd_circs_sim:
    result_dict = cd_sim_results.get_counts(c)
    avg_sm_cd_sim.append(staggered_magnetization(result_dict, shots))

In [ ]:
#Quantum processor post-processing for constant-depth circuits
cd_results = cd_job.result()        

avg_sm_cd_qp = []
for c in cd_circs_qp:
    result_dict = cd_results.get_counts(c)
    avg_sm_cd_qp.append(staggered_magnetization(result_dict, shots))

In [ ]:
#Post-processing for IBM circuit results from the simulator
avg_sm_ibm_sim = []
for c in ibm_circs_sim:
    result_dict = ibm_sim_results.get_counts(c)
    avg_sm_ibm_sim.append(staggered_magnetization(result_dict, shots))

In [ ]:
#Post-processing for IBM circuit results from the real quantum processor
ibm_results = ibm_job.result()        
avg_sm_ibm_qp = []
for c in ibm_circs_qp:
    result_dict = ibm_results.get_counts(c)
    avg_sm_ibm_qp.append(staggered_magnetization(result_dict, shots))

### Error Mitigation
We only perform minimal error mitigation to combat read-out error noise in the results from the real quantum hardware.  There will be no such noise on the quantum simulator as we chose to use noise-free simulators.  Given that we know the magnetization result for the initial time-step should be equal to 1, we scale all subsequent results by the initial magnetization measured in the first time-step.

In [ ]:
#error mitigation of readout-noise
#scaled all results by result from first time-step
cd_scaled = np.asarray(avg_sm_cd_qp)/avg_sm_cd_qp[0]
ibm_scaled = np.asarray(avg_sm_ibm_qp)/avg_sm_ibm_qp[0]

### Plot results

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(avg_sm_ibm_sim, label="Noise Free Simulation", color="blue")
plt.plot(ibm_scaled, label="IBM generated HW", color="green")
plt.plot(cd_scaled, label="Constant-Depth HW", color="red")
plt.plot(avg_sm_cd_sim, label="Constant-Depth Simulation", color="orange")
plt.legend()
plt.xlim(0,39)
plt.ylim(-1.0, 1.0)
plt.xlabel("Simulation Timestep", fontsize=18, fontname="Times New Roman")
plt.yticks([-1.0, -0.5, 0, 0.5, 1.0],fontsize=18, fontname="Times New Roman")
plt.xticks(fontsize=18, fontname="Times New Roman")
plt.ylabel("Average Magnetization", fontsize=18, fontname="Times New Roman")
plt.show()
#plt.savefig(f'{N}q_XY_cd_v_ibm.png.svg', dpi=500)

### Optionally write results from the quantum backend to file for future use

In [ ]:
#write all results to file for future use
with open(f'{N}q_XY_ibm_hardware.txt', 'w') as f:
    for item in avg_sm_ibm_qp:
        f.write("%s\n" % item)
with open(f'{N}q_XY_cd_hardware.txt', 'w') as f:
    for item in avg_sm_cd_qp:
        f.write("%s\n" % item)
with open(f'{N}q_XY_ibm_sim.txt', 'w') as f:
    for item in avg_sm_ibm_sim:
        f.write("%s\n" % item)
with open(f'{N}q_XY_cd_sim.txt', 'w') as f:
    for item in avg_sm_cd_sim:
        f.write("%s\n" % item)  